In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu May  7 07:09:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    31W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# # !pip install tensorflow-gpu==2.0.0-alpha0
# %tensorflow_version 2.x
# !pip uninstall -y tensorflow
# !pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io
import time
import numpy as np
import tensorflow as tf
import re
import time

In [0]:
path = '/content/gdrive/My Drive/Colab Notebooks/cmpe258/final-project/Version-v4/basic-seq2seq/'

In [0]:
import os
os.chdir(path)

In [7]:
!pwd

/content/gdrive/My Drive/Colab Notebooks/cmpe258/final-project/Version-v4/basic-seq2seq


In [0]:
conversations = open('data/dialogues_text.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
topics        = open('data/dialogues_topic.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')

In [9]:
print(len(conversations))
conversations[0:5]

13119


["The kitchen stinks . __eou__ I'll throw out the garbage . __eou__",
 'So Dick , how about getting some coffee for tonight ? __eou__ Coffee ? I don ’ t honestly like that kind of stuff . __eou__ Come on , you can at least try a little , besides your cigarette . __eou__ What ’ s wrong with that ? Cigarette is the thing I go crazy for . __eou__ Not for me , Dick . __eou__',
 'Are things still going badly with your houseguest ? __eou__ Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last straw . __eou__ Leo , I really think you ’ re beating around the bush with this guy . I know he used to be your best friend in college , but I really think it ’ s time to lay down the law . __eou__ You ’ re right . Everything is probably going to come to a

In [0]:
questions = []
answers   = []
for index in range(len(conversations)):
  _lines = conversations[index].split('__eou__')
  # print(_lines)
  _lines_length = len(_lines)
  for ind in range(_lines_length - 2):
    questions.append(_lines[ind])
    answers.append(_lines[ind + 1])

In [11]:
questions[0:10]

['The kitchen stinks . ',
 'So Dick , how about getting some coffee for tonight ? ',
 ' Coffee ? I don ’ t honestly like that kind of stuff . ',
 ' Come on , you can at least try a little , besides your cigarette . ',
 ' What ’ s wrong with that ? Cigarette is the thing I go crazy for . ',
 'Are things still going badly with your houseguest ? ',
 ' Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last straw . ',
 ' Leo , I really think you ’ re beating around the bush with this guy . I know he used to be your best friend in college , but I really think it ’ s time to lay down the law . ',
 'Would you mind waiting a while ? ',
 ' Well , how long will it be ? ']

In [12]:
answers[0:10]

[" I'll throw out the garbage . ",
 ' Coffee ? I don ’ t honestly like that kind of stuff . ',
 ' Come on , you can at least try a little , besides your cigarette . ',
 ' What ’ s wrong with that ? Cigarette is the thing I go crazy for . ',
 ' Not for me , Dick . ',
 ' Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last straw . ',
 ' Leo , I really think you ’ re beating around the bush with this guy . I know he used to be your best friend in college , but I really think it ’ s time to lay down the law . ',
 ' You ’ re right . Everything is probably going to come to a head tonight . I ’ ll keep you informed . ',
 ' Well , how long will it be ? ',
 " I'm not sure . But I'll get a table ready as fast as I can . "]

In [0]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"I ’ m", "i am", text)
    text = re.sub(r"don’t", "do not", text)
    text = re.sub(r"don ’ t", "do not", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"he ' s", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"she ' s", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"that ' s", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"what ' s", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"where ' s", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\' ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\' ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\' re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\' d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

In [14]:
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
print(clean_questions[0:5])
print(len(clean_questions))

['the kitchen stinks  ', 'so dick  how about getting some coffee for tonight  ', ' coffee  i do not honestly like that kind of stuff  ', ' come on  you can at least try a little  besides your cigarette  ', ' what ’ s wrong with that  cigarette is the thing i go crazy for  ']
89862


In [15]:
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))
print(clean_answers[0:5])
print(len(clean_answers))

[' i will throw out the garbage  ', ' coffee  i do not honestly like that kind of stuff  ', ' come on  you can at least try a little  besides your cigarette  ', ' what ’ s wrong with that  cigarette is the thing i go crazy for  ', ' not for me  dick  ']
89862


In [0]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())
  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  w = w.strip()
  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [0]:
def create_dataset(num_examples):
  sentence_pairs = []
  for index in range(num_examples):
    sentence_pair = [(preprocess_sentence(clean_questions[index])),preprocess_sentence(clean_answers[index])]
    sentence_pairs.append(sentence_pair)
  return zip(*sentence_pairs)

In [0]:
num_examples = 10000
inp_lang, targ_lang = create_dataset(num_examples)

In [0]:
general_lang = inp_lang + targ_lang

In [20]:
len(general_lang)

20000

In [0]:
def tokenize(inp_lang, targ_lang, general_lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(general_lang)
  input_tensor = lang_tokenizer.texts_to_sequences(inp_lang)
  input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, padding='post')
  target_tensor = lang_tokenizer.texts_to_sequences(targ_lang)
  target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, padding='post')
  return input_tensor, lang_tokenizer, target_tensor, lang_tokenizer

In [0]:
def load_dataset():
  input_tensor, inp_lang_tokenizer, target_tensor, targ_lang_tokenizer = tokenize(inp_lang, targ_lang, general_lang)
  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [0]:
# def tokenize(lang, general_lang):
#   lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
#   lang_tokenizer.fit_on_texts(general_lang)
  
#   tensor = lang_tokenizer.texts_to_sequences(lang)
#   tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
#   return tensor, lang_tokenizer

In [0]:
# def load_dataset():
#   input_tensor, inp_lang_tokenizer   = tokenize(inp_lang,  general_lang)
#   target_tensor, targ_lang_tokenizer = tokenize(targ_lang, general_lang)
#   return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [0]:
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset()

In [0]:
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [0]:
# input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.001)
input_tensor_train  = input_tensor
target_tensor_train = target_tensor

In [27]:
# print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))
print(len(input_tensor_train), len(target_tensor_train))

10000 10000


In [0]:
def id2word(lang):
  for t in range(1, 10):
    print ("%d ----> %s" % (t, lang.index_word[t]))

In [29]:
id2word(inp_lang)

1 ----> <start>
2 ----> <end>
3 ----> i
4 ----> you
5 ----> the
6 ----> it
7 ----> a
8 ----> to
9 ----> is


In [30]:
id2word(targ_lang)

1 ----> <start>
2 ----> <end>
3 ----> i
4 ----> you
5 ----> the
6 ----> it
7 ----> a
8 ----> to
9 ----> is


In [0]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [32]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[2])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[2])

Input Language; index to word mapping
1 ----> <start>
209 ----> coffee
3 ----> i
16 ----> do
43 ----> not
1945 ----> honestly
23 ----> like
17 ----> that
142 ----> kind
13 ----> of
729 ----> stuff
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
100 ----> come
32 ----> on
4 ----> you
21 ----> can
45 ----> at
774 ----> least
96 ----> try
7 ----> a
117 ----> little
534 ----> besides
30 ----> your
1420 ----> cigarette
2 ----> <end>


In [33]:
BUFFER_SIZE = len(input_tensor_train)
print("BUFFER_SIZE:", BUFFER_SIZE)
BATCH_SIZE = 8
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 128
units = 512
vocab_inp_size = len(inp_lang.word_index) +1
vocab_tar_size = len(targ_lang.word_index)+1
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

BUFFER_SIZE: 10000


In [34]:
example_input_batch, example_target_batch = next(iter(dataset))
print(example_input_batch)
example_input_batch_numpy = example_input_batch.numpy()
convert(inp_lang, example_input_batch_numpy[1])
example_input_batch.shape, example_target_batch.shape

tf.Tensor(
[[ 1 27  2 ...  0  0  0]
 [ 1  3 29 ...  0  0  0]
 [ 1  3 28 ...  0  0  0]
 ...
 [ 1 73  2 ...  0  0  0]
 [ 1 28  4 ...  0  0  0]
 [ 1 27  3 ...  0  0  0]], shape=(8, 147), dtype=int32)
1 ----> <start>
3 ----> i
29 ----> am
86 ----> sorry
35 ----> but
34 ----> there
12 ----> s
39 ----> no
33 ----> one
38 ----> here
93 ----> by
17 ----> that
191 ----> name
2 ----> <end>


(TensorShape([8, 147]), TensorShape([8, 147]))

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [36]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (8, 147, 512)
Encoder Hidden state shape: (batch size, units) (8, 512)


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)
    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))
    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)
    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)
    return context_vector, attention_weights

In [41]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (8, 512)
Attention weights shape: (batch_size, sequence_length, 1) (8, 147, 1)


In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [43]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden, sample_output)
print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (8, 6382)


In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)
  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  return tf.reduce_mean(loss_)

In [0]:
checkpoint_dir = 'cp-attention-single-layer-daily-dialog/'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
      loss += loss_function(targ[:, t], predictions)
      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [0]:
EPOCHS = 50
for epoch in range(EPOCHS):
  start = time.time()
  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0
  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss
    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,batch, batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    print()
    checkpoint.save(file_prefix = checkpoint_prefix)
  print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.3278
Epoch 1 Batch 100 Loss 0.3920
Epoch 1 Batch 200 Loss 0.3219
Epoch 1 Batch 300 Loss 0.5668
Epoch 1 Batch 400 Loss 0.3420
Epoch 1 Batch 500 Loss 0.3590
Epoch 1 Batch 600 Loss 0.4350
Epoch 1 Batch 700 Loss 0.3338
Epoch 1 Batch 800 Loss 0.4118
Epoch 1 Batch 900 Loss 0.5791
Epoch 1 Batch 1000 Loss 0.3331
Epoch 1 Batch 1100 Loss 0.3507
Epoch 1 Batch 1200 Loss 0.2736
Epoch 1 Loss 0.4036
Time taken for 1 epoch 613.5124502182007 sec

Epoch 2 Batch 0 Loss 0.5672
Epoch 2 Batch 100 Loss 0.3537
Epoch 2 Batch 200 Loss 0.5120
Epoch 2 Batch 300 Loss 0.2866
Epoch 2 Batch 400 Loss 0.2557
Epoch 2 Batch 500 Loss 0.4587
Epoch 2 Batch 600 Loss 0.3663
Epoch 2 Batch 700 Loss 0.2661
Epoch 2 Batch 800 Loss 0.2339
Epoch 2 Batch 900 Loss 0.2792
Epoch 2 Batch 1000 Loss 0.2898
Epoch 2 Batch 1100 Loss 0.2293
Epoch 2 Batch 1200 Loss 0.2036

Epoch 2 Loss 0.3490
Time taken for 1 epoch 402.82732796669006 sec

Epoch 3 Batch 0 Loss 0.3789
Epoch 3 Batch 100 Loss 0.3935
Epoch 3 Batch 200 Loss 0.4

In [0]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))
  sentence = preprocess_sentence(sentence)
  print(type(inp_lang))
  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
  inputs = tf.convert_to_tensor(inputs)
  result = ''
  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += targ_lang.index_word[predicted_id] + ' '
    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot
    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)
  return result, sentence, attention_plot

In [0]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [0]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  # attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  # plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [49]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [52]:
question = 'where are you from'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> where are you from <end>
Predicted translation: i am not sure <end> 
None


In [54]:
question = 'hi'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> hi <end>
Predicted translation: hi bill is it <end> 
None


In [55]:
question = 'what can I help you'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> what can i help you <end>
Predicted translation: i want my hair cut <end> 
None


In [56]:
question = 'may I help you'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> may i help you <end>
Predicted translation: yes fill it up please <end> 
None


In [58]:
question = 'what do you think about coffee'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> what do you think about coffee <end>
Predicted translation: it looks great <end> 
None


In [60]:
question = 'what do you think about the car'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> what do you think about the car <end>
Predicted translation: it s extremely expensive <end> 
None


In [64]:
question = 'do you like this short'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> do you like this short <end>
Predicted translation: yes how much is it <end> 
None


In [66]:
question = 'Can you sing a song'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> can you sing a song <end>
Predicted translation: sure what style do you need <end> 
None


In [68]:
question = 'I prefer a classic music'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> i prefer a classic music <end>
Predicted translation: it is not easy i am driving which is the problem <end> 
None


In [72]:
question = 'Thank you'
print(translate(question))

<class 'keras_preprocessing.text.Tokenizer'>
Input: <start> thank you <end>
Predicted translation: you are welcome please come again <end> 
None
